In [1]:
import sys
sys.path.append('..')

import json
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, auc

from egh_vlm.hallucination_dataset import load_features, hallucination_collate_fn, split_stratified
from egh_vlm.hallucination_detector import DetectorModule
from egh_vlm.training import eval_detector
from egh_vlm.utils import load_phd_dataset

## Analysis

In [2]:
detector = DetectorModule(2048, 2048, 1, 0.2)
detector.load_state_dict(torch.load('../data/phd/prototype/detector_full.pt'))
detector.eval()

DetectorModule(
  (fc1): Linear(in_features=2048, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=1, bias=True)
)

In [3]:
with open('../data/phd/prototype/phd_sample_qwen3_vl_2b.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)
features = load_features('../data/phd/prototype/features_full.pt')
hidden_size = 2048

In [4]:
train_dataset, val_dataset = split_stratified(
    features, train_ratio=0.7, random_state=42
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=16,
    collate_fn=hallucination_collate_fn,
    shuffle=True
)
test_dataloader = DataLoader(
    val_dataset,
    batch_size=16,
    collate_fn=hallucination_collate_fn,
    shuffle=True,
)

In [5]:
result = eval_detector(detector, test_dataloader)
print(f'Evaluation: ACC={result[0]}, F1={result[1]}, PR AUC={result[2]}')

Evaluation: ACC=0.7255075022065314, F1=0.6935960591133005, PR AUC=0.7973757872737853


## Dataset Construction

In [6]:
scored_dataset = []

with torch.no_grad():
    for batch_idx, batch in enumerate(test_dataloader):
        ids, embs, grads, labels = batch
        scores = detector(embs, grads).squeeze()
        
        for id, score in zip(ids, scores):
            scored_dataset.append({
                'id': id,
                'detector_score': float(score),
            })

In [7]:
test_dataset = []

for sample in scored_dataset:
    id = sample['id']
    data = next((s for s in dataset if s['id'] == id), None)
    if data is not None:
       data['detector_score'] = sample['detector_score']
       test_dataset.append(data)
    else:
        print(f'ID not found: {id}')
print(f'Length of test dataset: {len(test_dataset)}')

Length of test dataset: 1133


In [8]:
save_path = '../data/phd/prototype/phd_sample_qwen3_vl_2b_scored.json'

with open(save_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)